<a href="https://colab.research.google.com/github/k41n3w/EmpregaPop/blob/main/TCC_Predi%C3%A7%C3%A3o_de_Churn_de_Academia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.6 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=3a79dd06de2ad7c197656a1303f6d89334e1ed52f5cee6e0a1e6f35626388fe8
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin


In [ ]:
import pandas as pd

# Carregar o arquivo
df = pd.read_excel('/content/tcc_base.csv.xlsx')

# Visualizar as primeiras linhas
df.head()


,ID_REGISTRO,ID_CLIENTE,DATA_PRESENCA,ID_TREINADOR,ID_UNIDADE,ID_PLANO,PRESENCA
0,5093,73,2020-01-15,13,5,1,1
1,5193,73,2020-01-22,13,5,1,1
2,5293,73,2020-01-29,13,5,1,0
3,5393,73,2020-02-05,13,5,1,1
4,5493,73,2020-02-13,13,5,1,1


In [ ]:
from ydata_profiling import ProfileReport

# Criar o perfil
profile = ProfileReport(df, title="Relatório Exploratória - TCC Churn", explorative=True)

# Salvar o relatório em HTML
profile.to_file("relatorio_churn.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 7/7 [00:00<00:00, 57.23it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

# Carregar o CSV exportado
df_clientes = pd.read_csv('/content/clientes_apenas.csv')


In [ ]:
from ydata_profiling import ProfileReport

# Criar o relatório exploratório
profile = ProfileReport(df_clientes, title="Relatório de Clientes", explorative=True)

# Exportar como HTML
profile.to_file("relatorio_clientes.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 12/12 [00:00<00:00, 17.76it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from google.colab import files

files.download("relatorio_clientes.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =========================================
# 1. Instalar dependências
# =========================================
!pip install scikit-learn==1.4.2 pandas==2.2.2 numpy==1.26.4 joblib==1.4.2 --quiet

# =========================================
# 2. Imports e carregamento
# =========================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import joblib

# ► Ajuste o caminho se necessário (Colab → /content/…)
df = pd.read_csv('/content/clientes_apenas.csv')

# =========================================
# 3. FEATURE ENGINEERING
# =========================================
# A) Variável-alvo (churn / cancelado)
df['cancelado'] = (df['SN_ATIVO'].str.upper() == 'N').astype(int)

# B) Selecionar colunas
num_cols = ['IDADE']                      # numéricas
cat_cols = ['ID_TREINADOR', 'ID_UNIDADE', # categóricas
            'HORARIO', 'ID_TIPO_PLANO']

X = df[num_cols + cat_cols]
y = df['cancelado']

# =========================================
# 4. Treino / validação
# =========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# =========================================
# 5. Pipeline + Random Forest
# =========================================
numeric_transformer   = Pipeline([('scaler', StandardScaler())])
categorical_transform = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    [('num', numeric_transformer, num_cols),
     ('cat', categorical_transform, cat_cols)]
)

rf = RandomForestClassifier(
    n_estimators=400,
    min_samples_leaf=3,
    n_jobs=-1,
    class_weight='balanced',
    random_state=42
)

clf = Pipeline([('prep', preprocessor),
               ('rf'  , rf)])

clf.fit(X_train, y_train)

# =========================================
# 6. Avaliação
# =========================================
proba = clf.predict_proba(X_test)[:, 1]
pred  = clf.predict(X_test)

print(f'ROC-AUC: {roc_auc_score(y_test, proba):.4f}\n')
print(classification_report(y_test, pred, digits=4))
print('Matriz de Confusão:\n', confusion_matrix(y_test, pred))

# =========================================
# 7. Salvar o modelo
# =========================================
joblib.dump(clf, 'random_forest_churn.pkl')
print('Modelo salvo em random_forest_churn.pkl')


ROC-AUC: 0.5101

              precision    recall  f1-score   support

           0     0.4938    0.4790    0.4863      1334
           1     0.5307    0.5455    0.5380      1441

    accuracy                         0.5135      2775
   macro avg     0.5123    0.5122    0.5121      2775
weighted avg     0.5130    0.5135    0.5131      2775

Matriz de Confusão:
 [[639 695]
 [655 786]]
Modelo salvo em random_forest_churn.pkl
